In [1]:
import os

In [2]:
%pwd

"c:\\Users\\pc\\OneDrive - Institut National de Statistique et d'Economie Appliquee\\Documents\\GitHub\\Text-Summarization-NLP-Project\\research"

In [3]:
os.chdir("../")

In [4]:
%pwd

"c:\\Users\\pc\\OneDrive - Institut National de Statistique et d'Economie Appliquee\\Documents\\GitHub\\Text-Summarization-NLP-Project"

In [1]:
from pathlib import Path
from dataclasses import dataclass   

@dataclass(frozen=True)   
class ModelTrainerConfig : 
    root_dir : Path
    data_path: Path
    model_name : Path
    num_train_epochs : int
    warmup_steps : int
    per_device_train_batch_size : int
    per_device_eval_batch_size : int
    weight_decay : float
    logging_steps : int
    evaluation_strategy : str
    eval_steps : int
    save_steps : float
    gradient_accumulation_steps : int

In [3]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:

    # constructor initialised by config & params filepaths
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
    #Read yaml files & return it's content

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    #Create dirctory artfacts_root => for "config.artifacts_root" here is the use of ConfigBox
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig :
          config = self.config.model_trainer
          params = self.params.TrainingArguments
          create_directories([config.root_dir])

          model_trainer_config = ModelTrainerConfig(
                root_dir=config.root_dir ,
                data_path=config.data_path,
                model_name= config.model_name ,
                num_train_epochs= params.num_train_epochs,
                warmup_steps=params.warms_steps,
                per_device_train_batch_size=params.per_device_train_batch_size ,
                weight_decay=params.weight_decay,
                logging_steps=params.logging_steps,
                evaluation_strategy=params.eval_strategy ,
                eval_steps=params.evall_steps,
                save_steps=params.save_steps,
                gradient_accumulation_steps=params.gradient_accumulation_steps
          )



In [5]:
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
from datasets import load_dataset, load_from_disk
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

RuntimeError: Failed to import transformers.training_args because of the following error (look up to see its traceback):
cannot import name 'OrderedDict' from 'typing' (c:\Users\pc\AppData\Local\Programs\Python\Python37\lib\typing.py)

In [10]:
class ModelTrainer : 
    def __init__(self , config : ModelTrainerConfig) :
        self.config = config

    def train(self) : 
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_name)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_name).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #Loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)


        trainer_args = TrainingArguments(
            output_dir='pegasus-samsum', num_train_epochs=3, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        )
        trainer = Trainer(model=model_pegasus, args=trainer_args,
                        tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                        train_dataset=dataset_samsum_pt["train"], 
                        eval_dataset=dataset_samsum_pt["validation"])
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained("pegasus-samsum-model")
        ## Save tokenizer
        tokenizer.save_pretrained("tokenizer")


In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config= config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e